# Applied Data Science Capstone

## IBM Data Science 

### By Stephen Ewing

This is my write up for IBM's Applied Data Science Capstone.  

## Get the SOUPs

First I'll use the BeautifulSoup package to scrape the table from this Wikipedia page.  <https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M>

In [24]:
import requests
website_url = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text

from bs4 import BeautifulSoup
soup = BeautifulSoup(website_url, 'lxml')

## Make the PANDAs

This will snag all the table parts and put them together into a big ol' list.  The it'll get rid of the carrage returns.  Fianlly, it will grab every 3rd item to make each of the 3 columns.

In [25]:
list_thingy = []
for tr in soup.findAll("table"):
     for td in tr.find_all("td"):
         if not td.attrs.get('style'):
             list_thingy.append(td.text)

no_returns = [i.split('\n', 1)[0] for i in list_thingy]
col1 = no_returns[0::3]
del(col1[-1])
col2 = no_returns[1::3]
col3 = no_returns[2::3]

import pandas as pd
zip_list = pd.DataFrame({'Postcode': col1, 'Borough': col2, 'Neighbourhood': col3})
zip_list.head(11)

,Borough,Neighbourhood,Postcode
0,Not assigned,Not assigned,M1A
1,Not assigned,Not assigned,M2A
2,North York,Parkwoods,M3A
3,North York,Victoria Village,M4A
4,Downtown Toronto,Harbourfront,M5A
5,Downtown Toronto,Regent Park,M5A
6,North York,Lawrence Heights,M6A
7,North York,Lawrence Manor,M6A
8,Queen's Park,Not assigned,M7A
9,Not assigned,Not assigned,M8A


## Cleaning Part 1

This gets rid of the unassigned boroughs.

In [26]:
no_unassined_boroughs = zip_list[zip_list['Borough'] != "Not assigned"]
no_unassined_boroughs.head(11)

,Borough,Neighbourhood,Postcode
2,North York,Parkwoods,M3A
3,North York,Victoria Village,M4A
4,Downtown Toronto,Harbourfront,M5A
5,Downtown Toronto,Regent Park,M5A
6,North York,Lawrence Heights,M6A
7,North York,Lawrence Manor,M6A
8,Queen's Park,Not assigned,M7A
10,Etobicoke,Islington Avenue,M9A
11,Scarborough,Rouge,M1B
12,Scarborough,Malvern,M1B


## Cleaning Part 2

When the neighborhood doesn't have a name but the bourough does make the neighbourhood name the borough name.

In [22]:
import numpy as np
no_unassined_neighborhoods = no_unassined_boroughs.copy()
no_unassined_neighborhoods['Neighbourhood'] = np.where(no_unassined_boroughs['Neighbourhood'] == 'Not assigned', no_unassined_boroughs.Borough, no_unassined_boroughs.Neighbourhood)
no_unassined_neighborhoods.head(11)

,Borough,Neighbourhood,Postcode
2,North York,Parkwoods,M3A
3,North York,Victoria Village,M4A
4,Downtown Toronto,Harbourfront,M5A
5,Downtown Toronto,Regent Park,M5A
6,North York,Lawrence Heights,M6A
7,North York,Lawrence Manor,M6A
8,Queen's Park,Queen's Park,M7A
10,Etobicoke,Islington Avenue,M9A
11,Scarborough,Rouge,M1B
12,Scarborough,Malvern,M1B


## Cleaning Part 3

This groups the neighbourhoods that are in the same postcode together separated by columns.

In [23]:
grouped_postcodes = no_unassined_boroughs.groupby('Postcode')['Neighbourhood'].agg(lambda x: "%s" % ', '.join(x))
grouped_postcodes.head()

Postcode
M1B                            Rouge, Malvern
M1C    Highland Creek, Rouge Hill, Port Union
M1E         Guildwood, Morningside, West Hill
M1G                                    Woburn
M1H                                 Cedarbrae
Name: Neighbourhood, dtype: object